# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Импортируем файл и изучим общую информацию

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

display(df.head(20))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Из полученных данных видно что:
1. имеются пропущенные значения в столбцах 'days_employed' и 'total_income'
2. в столбце 'days_employed' есть как положительные так и отрицательные значения.
3. присутствует разный регистр в данных столбца 'education'
4. столбец 'purpose' потребуется разбить на категории

### Шаг 2.1 Заполнение пропусков

Выведем данные cо строками с пропущенными данными:

In [4]:
df[df['total_income'].isna() == True].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Отсутствующие значения в столбцах 'days_employed' и 'total_income' совпадают по строкам, проверим это:

In [5]:
df.loc[(df['total_income'].isna() == True) & (df['days_employed'].isna() == False)].sum()

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

In [6]:
'{:.1%}'.format(len(df[df['total_income'].isna() == True]) / len(df))

'10.1%'

Отсутствует 10 процентов данных, что существенно повлияет на результат исследования. Потребуется заполнить эти пропуски данных.
Оба столбца имеют тип float64, оценим диапазоны значений в этих столбцам

In [7]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [8]:
display(df['days_employed'].max())
display(df['days_employed'].min())
display(df['total_income'].max())
display(df['total_income'].min())

401755.40047533

-18388.949900568383

2265604.028722744

20667.26379327158

данные в столбце 'total_income' можно заполнить медианными значениям.
в столбце days_employed — общий трудовой стаж в днях, имеются отрицательные значения, определим какие строки соответствуют отрицательным значениям а какие положительным:

In [9]:
df.loc[(df['days_employed'] > 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [10]:
df[df['days_employed'] > 0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [11]:
df[df['days_employed'] < 0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Положительные значения присвоены безработным и пенсионерам.
Присвоим этим ячейкам пустые значения, а отрицательные значения переведем в положительные

In [12]:
df.loc[df['days_employed'] > 0, 'days_employed'] = None
df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df['days_employed'])

In [13]:
display(df['days_employed'].max())
display(df['days_employed'].min())
display(df['total_income'].max())
display(df['total_income'].min())

18388.949900568383

24.14163324048118

2265604.028722744

20667.26379327158

Заполним пропуски медианными значениями:

In [14]:
df['days_employed'].fillna(df['days_employed'].median(), inplace=True)

In [15]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))

In [16]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски данных устранены

### Шаг 2.2 Проверка данных на аномалии и исправления.

После общей оценки было замечено что:<br/> 
-присутствует разный регистр в данных столбца 'education'<br/> 
-в столбце 'purpose' имеются неявные дубликаты.<br/> 
Кроме того проведем проверку на явные дубликаты и на наличие аномальных данных и устраним их

Оценим значения в стобцах, для определения неявных дубликатов и аномальных значений

In [17]:
for n in df.columns:
    print(n, df[n].unique())
    print('-----')

children [ 1  0  3  2 -1  4 20  5]
-----
days_employed [8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]
-----
dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
-----
education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
-----
education_id [0 1 2 3 4]
-----
family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
-----
family_status_id [0 1 2 3 4]
-----
gender ['F' 'M' 'XNA']
-----
income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
-----
debt [0 1]
-----
total_income [253875.6394526  112080.01410244 145885.95229686 ...  89672.5

в столбце 'children' присутсвует отрицательное значение, и малореальное 20, проверим какой процент данных они занимают

In [18]:
'{:.1%}'.format(df[(df['children'] == -1) | (df['children'] == 20)]['children'].count() / len(df['children']))

'0.6%'

Процент незначительный, поэтому исключим эти строки:

In [19]:
df = df.drop(df[(df['children'] == -1) | (df['children'] == 20)]['children'].index, axis=0).reset_index(drop=True)

Приведем к единому регистру столбец 'education'

In [20]:
df['education'] = df['education'].str.lower()

в столбце 'dob_years' — возраст клиента в годах имеется значение 0, проверим процент этих значений

In [21]:
'{:.1%}'.format(df[df['dob_years'] == 0]['dob_years'].count() / len(df['dob_years']))

'0.5%'

Процент незначительный, поэтому исключим эти строки

In [22]:
df = df.drop(df[df['dob_years'] == 0]['dob_years'].index, axis=0).reset_index(drop=True)

в столбце 'gender' имеется значение 'XNA'

In [23]:
'{:%}'.format(df[df['gender'] == 'XNA']['gender'].count() / len(df['gender']))

'0.004694%'

In [24]:
df = df.drop(df[df['gender'] == 'XNA']['gender'].index, axis=0).reset_index(drop=True)

Приведем к единому регистру столбец family_status

In [25]:
df['family_status'] = df['family_status'].str.lower()

### Шаг 2.3. Изменение типов данных.

Изменим тип данных в столбцах total_income, dob_years, days_employed в тип int64 (целочисленный)

In [26]:
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

### Шаг 2.4. Удаление дубликатов.

в части проверки данных на аномалии, данные таблицы были приведены к единому формату.
выполним проверку на явные дубликаты после обработки таблицы.

In [27]:
df.duplicated().sum()

71

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

In [29]:
df.duplicated().sum()

0

Дубликаты устранены.
Причинами дубликатов изначально были явные дубликаты в таблице, кроме того дубликаты появились после приведения данных в столбцах таблицы к единому формату.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [30]:
df_education_id = pd.DataFrame({'education': df['education'], 'education_id': df['education_id']})
df_family_status_id = pd.DataFrame({'family_status': df['family_status'], 'family_status_id': df['family_status_id']})
df = df.drop(['education', 'family_status'], axis=1)
display(df_education_id)
display(df_family_status_id)
display(df.head(20))

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21225,среднее,1
21226,среднее,1
21227,среднее,1
21228,среднее,1


,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21225,гражданский брак,1
21226,женат / замужем,0
21227,гражданский брак,1
21228,женат / замужем,0


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Создадим и применим функцию для категоризации столбца 'total_income' по уровню дохода, добавим полученный стоолбец в таблицу df:

In [31]:
def income_cat(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif 1000001  <= income:
        return 'A'

    

In [32]:
df['total_income_category'] = df['total_income'].apply(income_cat)

### Шаг 2.7. Категоризация целей кредита.

Создадим и применим функцию для категоризации столбца 'purpose' цели получения кредита, добавим полученный стоолбец в таблицу df:

In [33]:
def purpose_cat(purpose):
    if ('недвиж' in purpose) or 'жиль' in purpose:
         return 'операции с недвижимостью'
    elif 'автомобил' in purpose:
         return 'операции с автомобилем'   
    elif 'свадьб' in purpose:
         return 'проведение свадьбы'
    elif 'образован' in purpose:
         return 'получение образования'

In [34]:
df['purpose_category'] = df['purpose'].apply(purpose_cat)

отобразим финальную таблицу

In [35]:
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21225,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21226,0,1630,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21227,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21228,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Ответы на вопросы.

создадим сводную таблицу для оценки влияния количества детей на выплату кредита, и оценим полученные данные

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [36]:
children_pivot = df.pivot_table(index=['debt'], columns='children', values = 'total_income', aggfunc='count')
children_pivot = children_pivot.fillna(0)
display(children_pivot)
print('процент возврата кредита при отсутсвии детей {:.1f}% '.format(children_pivot[0][0]/(children_pivot[0][0]+children_pivot[0][1])*100))
print('процент возврата кредита при наличии 1 ребенка {:.1f}% '.format(children_pivot[1][0]/(children_pivot[1][0]+children_pivot[1][1])*100))
print('процент возврата кредита при наличии 2 детей {:.1f}% '.format(children_pivot[2][0]/(children_pivot[2][0]+children_pivot[2][1])*100))
print('процент возврата кредита при наличии 3 детей {:.1f}% '.format(children_pivot[3][0]/(children_pivot[3][0]+children_pivot[3][1])*100))
print('процент возврата кредита при наличии 4 детей {:.1f}% '.format(children_pivot[4][0]/(children_pivot[4][0]+children_pivot[4][1])*100))
print('процент возврата кредита при наличии 5 детей {:.1f}% '.format(children_pivot[5][0]/(children_pivot[5][0]+children_pivot[5][1])*100))


children,0,1,2,3,4,5
debt,,,,,,
0,12963.0,4351.0,1845.0,301.0,37.0,9.0
1,1058.0,441.0,194.0,27.0,4.0,0.0


процент возврата кредита при отсутсвии детей 92.5% 
процент возврата кредита при наличии 1 ребенка 90.8% 
процент возврата кредита при наличии 2 детей 90.5% 
процент возврата кредита при наличии 3 детей 91.8% 
процент возврата кредита при наличии 4 детей 90.2% 
процент возврата кредита при наличии 5 детей 100.0% 


##### Вывод 1:

- С большей вероятностью выплачиают кредит в срок люди без детей (кроме случев с наличием 5 детей, по этой категории слишком малая выборка данных, чтобы оценить надежность этой категории требуется больше данных)
- Наличие детей снижает вероятность выплаты кредита в срок (на 0,7 - 2,3 процента)
- Чаще берут кредит люди с меньшим колчичеством детей. 
- При наличимм 3 детей и более количество взятых кредитов резко сокращается.
- Подавляющее число заемщиков не имеет детей.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Верно! 
</div>

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

создадим сводную таблицу для оценки влияния семейного положения на выплату кредита, и оценим полученные данные

In [37]:
family_status_pivot = df.pivot_table(index=['debt'], columns='family_status_id', values = 'children',aggfunc='count')
display(family_status_pivot)
print('процент возврата кредита в семейном статусе женат / замужем {:.1f}% '.format(family_status_pivot[0][0]/(family_status_pivot[0][0]+family_status_pivot[0][1])*100))
print('процент возврата кредита в семейном статусе гражданский брак {:.1f}% '.format(family_status_pivot[1][0]/(family_status_pivot[1][0]+family_status_pivot[1][1])*100))
print('процент возврата кредита в семейном статусе вдовец / вдова {:.1f}% '.format(family_status_pivot[2][0]/(family_status_pivot[2][0]+family_status_pivot[2][1])*100))
print('процент возврата кредита в семейном статусе в разводе {:.1f}% '.format(family_status_pivot[3][0]/(family_status_pivot[3][0]+family_status_pivot[3][1])*100))
print('процент возврата кредита в семейном статусе не женат / не замужем {:.1f}% '.format(family_status_pivot[4][0]/(family_status_pivot[4][0]+family_status_pivot[4][1])*100))



family_status_id,0,1,2,3,4
debt,,,,,
0,11290,3729,884,1095,2508
1,923,383,62,84,272


процент возврата кредита в семейном статусе женат / замужем 92.4% 
процент возврата кредита в семейном статусе гражданский брак 90.7% 
процент возврата кредита в семейном статусе вдовец / вдова 93.4% 
процент возврата кредита в семейном статусе в разводе 92.9% 
процент возврата кредита в семейном статусе не женат / не замужем 90.2% 


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Согласен! Идём дальше)
</div>

##### Вывод 2:

##### Вопрос 3: Как разные цели кредита влияют на его возврат в срок?

- чаще всего креди в срок выплачивают люди в семейном статусе вдовец / вдова, в статусе в разводе или в статусе женат / замужем, среднем на 2,5 процента чаще чем в статусе гражданский брак и не женат / не замужем
- БОльшую часть заемщиков составляют заемщики в статусе женат / замужем.
- Реже всего кредит берут в семейном статусе вдовец / вдова и в статусе в разводе

создадим сводную таблицу для оценки влияния уровня дохода на выплату кредита, и оценим полученные данные

In [38]:
total_income_pivot = df.pivot_table(index=['debt'], columns='total_income_category', values = 'total_income',aggfunc='count')
display(total_income_pivot)

print('процент возврата кредита при уровне дохода 0–30000 {:.1f}% '.format(total_income_pivot['A'][0]/(total_income_pivot['A'][0]+total_income_pivot['A'][1])*100))
print('процент возврата кредита при уровне дохода 30001–50000  {:.1f}% '.format(total_income_pivot['B'][0]/(total_income_pivot['B'][0]+total_income_pivot['B'][1])*100))
print('процент возврата кредита при уровне дохода 50001–200000 {:.1f}% '.format(total_income_pivot['C'][0]/(total_income_pivot['C'][0]+total_income_pivot['C'][1])*100))
print('процент возврата кредита при уровне дохода 200001–1000000 {:.1f}% '.format(total_income_pivot['D'][0]/(total_income_pivot['D'][0]+total_income_pivot['D'][1])*100))
print('процент возврата кредита при уровне дохода 1000001 и выше {:.1f}% '.format(total_income_pivot['E'][0]/(total_income_pivot['E'][0]+total_income_pivot['E'][1])*100))

total_income_category,A,B,C,D,E
debt,,,,,
0,23,4634,14503,326,20
1,2,353,1346,21,2


процент возврата кредита при уровне дохода 0–30000 92.0% 
процент возврата кредита при уровне дохода 30001–50000  92.9% 
процент возврата кредита при уровне дохода 50001–200000 91.5% 
процент возврата кредита при уровне дохода 200001–1000000 93.9% 
процент возврата кредита при уровне дохода 1000001 и выше 90.9% 


##### Вывод 3:

- люди с самым низким доходом (0–30000) и с самым высоким (1000001 и выше) практические не берут кредиты
- чаще всего выплачивают кредит в срок люди с доходом 200001–1000000
- основная часть заемщиков имеет доход 30001–200000, при увеличении дохода в этих границах процент выплаты кредита в срок снижается

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

создадим сводную таблицу для оценки зависимости цели займа на выплату кредита, и оценим полученные данные

In [39]:
total_income_pivot = df.pivot_table(index=['debt'], columns='purpose_category', values = 'purpose',aggfunc='count')
display(total_income_pivot)
print('процент возврата кредита при цели займа: операции с автомобилем  {:.1f}% '.format(total_income_pivot['операции с автомобилем'][0]/(total_income_pivot['операции с автомобилем'][0]+total_income_pivot['операции с автомобилем'][1])*100))
print('процент возврата кредита при цели займа: операции с недвижимостью {:.1f}% '.format(total_income_pivot['операции с недвижимостью'][0]/(total_income_pivot['операции с недвижимостью'][0]+total_income_pivot['операции с недвижимостью'][1])*100))
print('процент возврата кредита при цели займа: получение образования {:.1f}% '.format(total_income_pivot['получение образования'][0]/(total_income_pivot['получение образования'][0]+total_income_pivot['получение образования'][1])*100))
print('процент возврата кредита при цели займа: проведение свадьбы  {:.1f}% '.format(total_income_pivot['проведение свадьбы'][0]/(total_income_pivot['проведение свадьбы'][0]+total_income_pivot['проведение свадьбы'][1])*100))

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,,,,
0,3861,9926,3601,2118
1,397,777,369,181


процент возврата кредита при цели займа: операции с автомобилем  90.7% 
процент возврата кредита при цели займа: операции с недвижимостью 92.7% 
процент возврата кредита при цели займа: получение образования 90.7% 
процент возврата кредита при цели займа: проведение свадьбы  92.1% 


##### Вывод 4:

- Люди чаще выплачивают кредит в срок если он взят на операции недвижимостью или на проведение свадьбы.
- БОльшая часть кредитов берется на операции с недвижимостью

## Общий вывод:

Была выполнена обработка исходных данных в ходе которой были устранены аномальные данные, значения по столбцам приведены к единому формату, были удалены дубликаты, выполнена категоризация данных.

По результатам исследования наиболее кредитоспособными клиентами являются:
- клиенты без детей
- клиенты с доходом 200001–1000000
- в семейном статусе вдовец / вдова
- с целью займа на операции с недвижимостью